<a href="https://colab.research.google.com/github/Satwikram/Transformer-model-for-language-understanding/blob/master/%20Portuguese%20to%20English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Author: Satwik Ram K
Portuguese to English Transalation

## Importing Dependencies

In [4]:
import numpy as np
import pandas as pd
import re
import math
import re
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt


## Loading Dataset

The dataset is available in Tfds

In [5]:
data, metadata = tfds.load('ted_hrlr_translate/pt_to_en',with_info = True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteXWILNN/ted_hrlr_translate-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteXWILNN/ted_hrlr_translate-validation.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteXWILNN/ted_hrlr_translate-test.tfrecord


Dataset ted_hrlr_translate downloaded and prepared to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0. Subsequent calls will reuse this data.


In [6]:
data

{'test': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.string)>,
 'train': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.string)>,
 'validation': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.string)>}

In [7]:
metadata

tfds.core.DatasetInfo(
    name='ted_hrlr_translate',
    version=1.0.0,
    description='Data sets derived from TED talk transcripts for comparing similar language pairs
where one is high resource and the other is low resource.
',
    homepage='https://github.com/neulab/word-embeddings-for-nmt',
    features=Translation({
        'en': Text(shape=(), dtype=tf.string),
        'pt': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=54781,
    splits={
        'test': 1803,
        'train': 51785,
        'validation': 1193,
    },
    supervised_keys=('pt', 'en'),
    citation="""@inproceedings{Ye2018WordEmbeddings,
      author  = {Ye, Qi and Devendra, Sachan and Matthieu, Felix and Sarguna, Padmanabhan and Graham, Neubig},
      title   = {When and Why are pre-trained word embeddings useful for Neural Machine Translation},
      booktitle = {HLT-NAACL},
      year    = {2018},
      }""",
    redistribution_info=,
)

## Taking Train and Validation Data

In [8]:
train, val = data['train'], data['validation']

In [11]:
type(train)

tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter

## Creating a custom subwords tokenizer from the training dataset.

In [21]:
count = 0
for a, b in train:
  count += 1
  if count == 10:
    break
  else:
    print("Portugees sentence:",a)
    print("English Corresponding Sentence",b)
    print("--"*60)
  

Portugees sentence: tf.Tensor(b'e quando melhoramos a procura , tiramos a \xc3\xbanica vantagem da impress\xc3\xa3o , que \xc3\xa9 a serendipidade .', shape=(), dtype=string)
English Corresponding Sentence tf.Tensor(b'and when you improve searchability , you actually take away the one advantage of print , which is serendipity .', shape=(), dtype=string)
------------------------------------------------------------------------------------------------------------------------
Portugees sentence: tf.Tensor(b'mas e se estes fatores fossem ativos ?', shape=(), dtype=string)
English Corresponding Sentence tf.Tensor(b'but what if it were active ?', shape=(), dtype=string)
------------------------------------------------------------------------------------------------------------------------
Portugees sentence: tf.Tensor(b'mas eles n\xc3\xa3o tinham a curiosidade de me testar .', shape=(), dtype=string)
English Corresponding Sentence tf.Tensor(b"but they did n't test for curiosity .", shape=(), 

In [22]:
tokenizer_en = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    (en.numpy() for pt, en in train), target_vocab_size = 2**13
)

tokenizer_pt = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    (pt.numpy() for pt, en in train), target_vocab_size = 2**13
)

In [23]:
tokenizer_en.save_to_file('tokenizer_en')
tokenizer_pt.save_to_file('tokenizer_pt')

In [24]:
# Loading saved tokenizer
encoder_en = tfds.features.text.SubwordTextEncoder.load_from_file('/content/tokenizer_en')
encoder_pt = tfds.features.text.SubwordTextEncoder.load_from_file('/content/tokenizer_pt')

In [27]:
sample_string = "Transfomer is cool."

encoded_string = encoder_en.encode(sample_string)

original_string = encoder_en.decode(encoded_string)

print("Orginal String:",sample_string)
print("Encoded String:",encoded_string)
print("Decoded String:",original_string)

Orginal String: Transfomer is cool.
Encoded String: [7915, 1248, 7946, 1391, 2762, 7863, 13, 1671, 7877]
Decoded String: Transfomer is cool.


In [28]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

## Add a start and end token to the input and target.